# DLT pipeline

This Delta Live Tables (DLT) definition is executed using a pipeline defined in resources/dlt_incremental_refresh_demo.pipeline.yml.

In [ ]:
# Import DLT and src/dlt_incremental_refresh_demo
import dlt
import sys

In [ ]:
# Add widgets for catalog and schema parameters
dbutils.widgets.text("catalog", "", "Catalog Name")
dbutils.widgets.text("schema", "", "Schema Name")

# Get parameter values (fallback to defaults if not provided)
catalog = dbutils.widgets.get("catalog") or spark.conf.get("bundle.var.catalog", "main")
schema = dbutils.widgets.get("schema") or spark.conf.get("bundle.var.schema", "incremental_dlt")

sys.path.append(spark.conf.get("bundle.sourcePath", "."))
from pyspark.sql.functions import expr
# from dlt_incremental_refresh_demo import main

In [ ]:
@dlt.table
def name_group_1():
    return spark.sql(f"""
        SELECT
            name,
            COUNT(id) AS name_count
        FROM {catalog}.{schema}.random_data
        GROUP BY name; """)

@dlt.table
@dlt.expect_all_or_drop({"valid_customer_name" : "name <> 'Alice'"})
def filtered_name():
    return dlt.read("name_group_1")

In [ ]:
@dlt.table
def name_group_2():
    return spark.sql(f"""
        SELECT
            name,
            CURRENT_TIMESTAMP() AS load_time,
            COUNT(id) AS name_count
        FROM {catalog}.{schema}.random_data
        GROUP BY ALL; """)

@dlt.table
def full_table():
    return spark.sql(f"""SELECT * FROM {catalog}.{schema}.random_data;""")